In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import collections

In [3]:
kroeber = pd.read_excel(f'../data/Kroeber texts.xlsx',None)
reichard = pd.read_excel(f'../data/Reichard texts.xlsx',None)
teeter = pd.read_excel(f'../data/Teeter texts.xlsx',None)

/Users/leonjdaeim/opt/miniconda3/envs/EDA/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [ ]:
# crow = wiyotData['Crow']
# skunk = wiyotData['Crow']
# pelican = wiyotData['Pelican']

In [4]:
def processLine(x):
    try:
        x = '\t'.join(x.dropna()).split('\t')
        return x
    except:
        return None
    
getlowercase = lambda x: x if x.isupper() else x.lower()    

def getWordPairs(phrase):
    wordpairlist = []
    for i in range(0,len(phrase)-1):
        wordpairlist.append('|'.join([phrase[i],phrase[i+1]]))
    return wordpairlist

def countFrequencyWordPairs(listofwordpairs):
    try:
        return collections.Counter(listofwordpairs[:])
    except:
        return None

In [5]:
_isna = lambda x: x.isna()

## Remove empty lines
import re
matchEndPeriod = re.compile(r'\.$')

def getWordPairsFromText(data):
    try:
        ### Remove empty lines
        data = data[~data.apply(_isna,0).all(1)]
        storyWordPairs = []
        for i in range(0,len(data),2):
            wiyotPhrase = data.iloc[i]
            englishPhrase = data.iloc[i+1]
            ####
            wiyotPhrase = processLine(wiyotPhrase)
            ### Do some extra processing to the english phrase
            englishPhrase = processLine(englishPhrase)
            #englishPhrase = [getlowercase(x) for x in englishPhrase]
            ### Remove EMP = empty, remove periods at the end of gloss
            englishPhrase = [getlowercase(matchEndPeriod.sub('',x)) for x in englishPhrase if x !='EMP'] #Remove EMP for empty 
            ###
            wps = getWordPairs(englishPhrase)
            storyWordPairs.append(wps)    
        storyWordPairs = sum(storyWordPairs,[])    
        return storyWordPairs
    except:
        return []

In [17]:
processLine(wiyotPhrase)

['mes', 'ákwi.', 'Hi', 'khá', 'thawól', 'ił.']

In [50]:
def getAllWordPairsFromAuthor(author):
    stories_dictionary = dict()
    for story, text in author.items():
        stories_dictionary[story] = getWordPairsFromText(text)
    return sum(stories_dictionary.values(),[])

In [63]:
kroeber_wordpairs = getAllWordPairsFromAuthor(kroeber)
reichard_wordpairs = getAllWordPairsFromAuthor(reichard)
teeter_wordpairs = getAllWordPairsFromAuthor(teeter)

---

In [64]:
all_authors_wordpairs = kroeber_wordpairs + reichard_wordpairs + teeter_wordpairs

In [65]:
all_biword_frequency = countFrequencyWordPairs(all_authors_wordpairs)

In [66]:
word_pair_frequency = pd.DataFrame.from_dict(all_biword_frequency,orient='index',columns=['frequency'])

In [67]:
word_pair_frequency = word_pair_frequency.reset_index()

In [68]:
word_pair_frequency.insert(0,'word_1',word_pair_frequency['index'].map(lambda x: x.split('|')[0]))
word_pair_frequency.insert(1,'word_2',word_pair_frequency['index'].map(lambda x: x.split('|')[1]))

In [69]:
word_pair_frequency = word_pair_frequency.rename(columns={"index":'biword'})
word_pair_frequency.to_csv('../results/word_pair_frequency.tsv',sep='\t',index=None)

In [70]:
word_pair_frequency.to_clipboard()